# Iterative Algorithm Design

### Sabryn Bley & Thomas Benzshawel
### CSC 3310 Lab 2
### Fall 2023

### Introduction: TODO

### Set-Up:

In [3]:
import random

### The "Decision Rule":

1. A paragraph describing a “decision rule” that can be applied to solve to the computational
problem. Provide at least 2 illustrations (test cases) that demonstrate how the rule is applied.

### Pseudocode:
2. High-level pseudocode for an algorithm that uses that rule to solve the computational problem
for any input

### Justification:
3. Provide an explanation and justification for why your algorithm is correct (1-3 paragraphs)

### Worst-Case Analysis:
4. Perform an analysis of the worst-case run time using asymptotic notation.

### The Code:
7. Attach all of your source code and test cases in an appendix

The following cell defines a helper function "is_problematic". This function accepts three points (at a time) that make up the polygon. This function calculates the cross product for the vectors the points make up and check if they are collinear. It also checks if the points are clockwise. If either of these are true, the points are problematic and violate a simple, closed polygon. If the points are valid (false), is returned.

In [153]:
def is_problematic(p1, p2, p3):
    '''
    This function checks whether three points violate making a simple, closed polygon.
    If the points lie on the same line using vector cross product or if they are 
    clockwise, the points violate the simple, closed polygon. If the points help
    make a simple, closed polygon are returned.
    
    :param p1: The (x, y) coordinates representing the first point.
    :param p2: The (x, y) coordinates representing the middle point.
    :param p3: The (x, y) coordinates representing the last point.
    :return invalid: True if the points violate being a simple, closed polygon. False otherwise. 
    '''
    # CALCULATE THE CROSS PRODUCT 
    # (a, b) = v1 & (c, d) = v2
    cp1 = (p2[1] - p1[1]) * (p3[0] - p2[0]) # a * d
    cp2 = (p3[1] - p2[1]) * (p2[0] - p1[0]) # b * c
    
    # COLLINEAR IF DIFFERENCE IS 0
    invalid = cp1 == cp2

    # CHECK IF POINTS ARE CLOCKWISE
    if not invalid: # if points are still "valid" (do not want to overwrite cp)
        invalid = (p2[0] - p1[0]) * (p3[1] - p1[1]) - (p2[1] - p1[1]) * (p3[0] - p1[0]) <= 0
        
    # RETURN IF POINTS VIOLATE A SIMPLE, CLOSED POLYGON OR NOT
    return invalid
    

The following cell defines a helper function "create_random_polygon". This function generates random points (x, y) to create a polygon. This function calls the "is_problematic" helper function to check that all the points make a simple, closed polygon. If the points violate the polygon, they are removed. After the removal, it assures that there are at least points (required to make a polygon). If there aren't, the entire process is repeated until a simple, closed polygon is created. Then the points are returned. It is assumed the last point connects back to the first point in the list.

In [154]:
def create_random_polygon():
    """
    Generate a random simple, closed polygon by generating a list of points in order.
    This method restricts the polygons to have at least three and at most ten points.
    Additionally, this method restricts the polygons to have coordinates in the first quadrant (maxing at a value of ten). 
    It is assumed the last point connects back to the first point in the list.

    :return points: A list of tuples that represent (x, y) coordinates for each point in the polygon.
    """
    valid = False
    while not valid:
        num_points = random.randint(5, 10)

        # CREATE LIST TO SAVE CREATED POINTS
        points = []

        # CREATE RANDOM POINTS
        for i in range(num_points - 1): 
            x = random.randint(0, 10) # create random x-coordinate
            y = random.randint(0, 10) # create random y-coordinate
            points.append((x, y)) # save coordinates as a new point
        
        # ADD FIRST POINT TO CREATE CLOSED POLYGON
        points.append(points[0])
    

        # SORT POINTS BASED OFF OF X COORDINATE
        points.sort(key = lambda x: x[0])

        # CHECK FOR PROBLEMATIC POINTS
        i = 0
        while i < len(points) - 2:
            if is_problematic(points[i], points[i + 1], points[i + 2]):
                points.pop(i + 1) # Remove the middle point
            else:
                i += 1 # Move to next three points
        
        # CHECK THERE IS MORE THAN TWO POINTS
        if len(points) > 2:
            valid = True
    
    # RETURN POINTS
    return points

In [155]:
# TEST
random_polygon = create_random_polygon()
print(random_polygon)

[(1, 3), (2, 2), (4, 6), (5, 5), (8, 0), (9, 7)]


### Benchmarks:
7. A table of your test cases, the answers you expect, and the answers returned by running your
implementation of the algorithm.
6. A table and graph from benchmarking your implementation on problem instances of different
sizes. The benchmarks should support your theoretically-derived run time.

### Conclusion: